In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple, defaultdict
from pathlib import Path
from enum import Enum
from os.path import splitext

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

# Results

In [12]:
system = "perlmutter"
version = "v0.5.1"
oldversion = "v0.5.0"

## Current

In [13]:
new = analyze.Analysis(f"results/{system}/")
print(new)

Analysis for Celeritas v0.5.1 on perlmutter


In [14]:
analyze.calc_geo_frac(new)

count      mean       std
problem                     geo     arch                           
atlas-tilecal               orange  cpu     4.0  0.780288  0.009956
                            vecgeom cpu     4.0  0.620552  0.014639
cms-hgcal                   orange  cpu     4.0  0.699002  0.007664
                            vecgeom cpu     4.0  0.619777  0.011550
cms2018                     vecgeom cpu     4.0  0.687225  0.010661
cms2018+field+msc           vecgeom cpu     4.0  0.801510  0.010776
                                    gpu     4.0  0.975397  0.004530
testem15                    orange  cpu     4.0  0.419691  0.007109
testem15+field              orange  cpu     4.0  0.545521  0.013664
                                    gpu     4.0  0.583846  0.003896
testem15+field+msc          orange  cpu     4.0  0.640543  0.008169
                            vecgeom cpu     4.0  0.604929  0.015689
testem3-composite+field+msc orange  cpu     4.0  0.779797  0.011561
                                    gpu     4.0  0.866004  0.001732
                            vecgeom cpu     4.0  0.762860  0.008107
                                    gpu     4.0  0.899418  0.004522
testem3-composite+msc       orange  cpu     4.0  0.735976  0.018226
                            vecgeom cpu     4.0  0.698452  0.020946
testem3-expanded+field+msc  orange  cpu     4.0  0.769533  0.035663
                            vecgeom cpu     4.0  0.763461  0.017289
testem3-flat                orange  cpu     4.0  0.666029  0.008682
                            vecgeom cpu     4.0  0.515269  0.006617
testem3-flat+field          orange  cpu     4.0  0.721115  0.015189
testem3-flat+field+msc      orange  cpu     4.0  0.792670  0.022659
                                    gpu     4.0  0.868433  0.001866
                            vecgeom cpu     4.0  0.772973  0.008238
                                    gpu     4.0  0.931627  0.004568
testem3-flat+msc            orange  cpu     4.0  0.765956  0.021306

## Previous

In [15]:
old = analyze.Analysis(f"results-old/{system}/")
print(old, f"({oldversion})")

Analysis for Celeritas v0.5.0-dev.222+33193b7f2 on perlmutter (v0.5.0)


In [16]:
ftab = analyze.make_failure_table(old.failures())
ftab.to_frame()

,Failure


# Analysis

## Compare manually

In [38]:
def get_throughput(analysis):
    col = analysis.result['avg_event_per_time']
    return col.xs('vecgeom', level='geo').unstack('arch')

def get_slots_per_stream(analysis):
    col = analysis.input['num_track_slots']
    return col.xs('vecgeom', level='geo').unstack('arch')

def compare(func):
    return 

In [42]:
def get_num_streams(analysis):
    c = new.cpu_per_task
    return pd.Series({'g4': np.nan, 'cpu': c,  'gpu': 1, 'cpu+g4': c, 'gpu+g4': 1})

def get_total_slots(analysis):
    return get_num_streams(analysis) * get_slots_per_stream(analysis)

In [18]:
rel_throughput = get_throughput(new) / get_throughput(old) - 1

In [19]:
rel_throughput.groupby('problem').mean()

arch,cpu,cpu+g4,gpu,gpu+g4,gpu+sync
problem,,,,,
atlas-tilecal,-0.119749,0.494502,-0.048790,0.274572,NaN
cms-hgcal,-0.067068,0.438892,-0.026365,0.314050,NaN
cms2018,-0.203373,0.285384,-0.106369,0.387242,NaN
cms2018+field+msc,-0.060940,0.179298,-0.033797,2.031561,-0.034012
testem15+field,NaN,NaN,NaN,NaN,0.021797
testem15+field+msc,-0.029170,0.404604,0.018801,0.206697,NaN
testem3-composite+field+msc,0.019985,0.127116,-0.002590,0.935424,-0.000982
testem3-composite+msc,-0.008002,0.202127,-0.003779,0.349950,NaN
testem3-expanded+field+msc,0.013979,0.101351,-0.003755,0.942053,NaN


In [43]:
get_total_slots(new) / get_total_slots(old)

,cpu,cpu+g4,g4,gpu,gpu+g4,gpu+sync
problem,,,,,,
atlas-tilecal,1.0,0.0625,NaN,1.0,4.0,NaN
cms-hgcal,1.0,0.0625,NaN,1.0,4.0,NaN
cms2018+field+msc,1.0,0.0625,NaN,1.0,4.0,NaN
cms2018,1.0,0.0625,NaN,1.0,4.0,NaN
testem15+field+msc,1.0,0.0625,NaN,1.0,4.0,NaN
testem15+field,NaN,NaN,NaN,NaN,NaN,NaN
testem3-composite+field+msc,1.0,0.0625,NaN,1.0,4.0,NaN
testem3-composite+msc,1.0,0.0625,NaN,1.0,4.0,NaN
testem3-expanded+field+msc,1.0,0.0625,NaN,1.0,4.0,NaN


In [47]:
get_slots_per_stream(new).iloc[1]

arch
cpu           65536.0
cpu+g4         4096.0
gpu         1048576.0
gpu+g4       262144.0
gpu+sync          NaN
Name: cms-hgcal, dtype: float64

In [48]:
get_slots_per_stream(old).iloc[1]

arch
cpu           65536.0
cpu+g4        65536.0
gpu         1048576.0
gpu+g4        65536.0
gpu+sync          NaN
Name: cms-hgcal, dtype: float64

## Performance drop between versions

In [21]:
#analyses = {a.version: a for a in [frontier, frontier_v050]}
analyses = {
    oldversion: old,
    version: new,
}
throughput = {k: a.result['avg_event_per_time'] for k, a in analyses.items()}
plot_rel_to = old

In [22]:
def plot_ratio(result_dict):
    rel = analyze.summarize_instances(result_dict[version] / result_dict[oldversion])
    rel.dropna(inplace=True)
    fig, ax = plt.subplots(layout="constrained")
    plot_rel_to.plot_results(ax, rel)
    ax.grid(which='both')
    ax.legend()
    ax.text(0.98, 0.02,
        f"{version} / {oldversion} - 1 on {system}",
        va='bottom', ha='right',
        fontstyle='italic', color=(0.5,)*3, size='xx-small',
        transform=ax.transAxes,
        zorder=-100
    )

    return ax

In [23]:
steps = {k: a.result['avg_steps_per_primary'] for k, a in analyses.items()}

In [24]:
ax = plot_ratio(throughput)
ax.set_ylabel(f"Throughput [events/sec]")
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.pdf", transparent=True)
plt.close()

In [25]:
ax = plot_ratio(steps)
ax.set_ylabel(f"Work [steps/primary]")
ax.get_figure().savefig(f"results-old/{system}/rel-work.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-work.pdf", transparent=True)
plt.close()

In [27]:
!open results-old/{system}